In [1]:
#Import Dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import pandas and read the charity_data.csv from the provided cloud URL.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=["EIN", "NAME"])


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_type_counts = application_type_counts[application_type_counts < 500].index.tolist()
application_types_to_replace = application_type_counts

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# To look at CLASSIFICATION value counts >1
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts = classification_counts[classification_counts > 1] 
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_counts = application_df['CLASSIFICATION'].value_counts()
classification_counts = classification_counts[classification_counts < 100].index.tolist()   
classifications_to_replace = classification_counts

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      669
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
Name: count, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 
                                                        'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 
                                                        'SPECIAL_CONSIDERATIONS', 'INCOME_AMT'], drop_first=True)

application_df = application_df.astype(int)
application_df.head()


,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_Y,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6692,1,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [10]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)


In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [12]:
#Define Model
nn_optimized = tf.keras.models.Sequential()

#Layer 1
nn_optimized.add(tf.keras.layers.Dense(units=256, activation='relu', input_dim=X_train_scaled.shape[1]))

#Layer 2
nn_optimized.add(tf.keras.layers.Dense(units=128, activation='relu'))

#Layer 3
nn_optimized.add(tf.keras.layers.Dense(units=64, activation='relu'))

#Output Layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn_optimized.summary()

c:\Users\asif_\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 256)            │        11,008 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 52,225 (204.00 KB)

 Trainable params: 52,225 (204.00 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
#Compile Model
nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [14]:
#Train Model
fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)


Epoch 1/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7114 - loss: 0.5834 - val_accuracy: 0.7203 - val_loss: 0.5649
Epoch 2/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7294 - loss: 0.5553 - val_accuracy: 0.7226 - val_loss: 0.5632
Epoch 3/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7301 - loss: 0.5493 - val_accuracy: 0.7224 - val_loss: 0.5632
Epoch 4/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7310 - loss: 0.5505 - val_accuracy: 0.7248 - val_loss: 0.5629
Epoch 5/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7339 - loss: 0.5463 - val_accuracy: 0.7232 - val_loss: 0.5579
Epoch 6/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.7295 - loss: 0.5486 - val_accuracy: 0.7244 - val_loss: 0.5594
Epoch 7/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7340 - loss: 0.5461 - val_accuracy: 0.7252 - val_loss: 0.5583
Epoch 8/150
644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.7412 - loss: 0.5401 - val_accu

In [15]:
#Evaluate Model
loss, accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Loss: 0.7566525936126709, Accuracy: 0.7279300093650818


In [16]:
#Evaluate Model
loss, accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Loss: {loss}, Accuracy: {accuracy}")

Loss: 0.7566525936126709, Accuracy: 0.7279300093650818


In [ ]:

#Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#Scaling
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#Define Model
nn_optimized = tf.keras.models.Sequential()

#Layer 1
nn_optimized.add(tf.keras.layers.Dense(units=256, activation='relu', input_dim=X_train_scaled.shape[1]))

#Layer 2
nn_optimized.add(tf.keras.layers.Dense(units=128, activation='tanh')) #Changed activation function

#Layer 3
nn_optimized.add(tf.keras.layers.Dense(units=64, activation='relu'))

#Output Layer
nn_optimized.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

#Compile Model
nn_optimized.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#Train Model
fit_model = nn_optimized.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.2)


Epoch 1/200


c:\Users\asif_\anaconda3\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


644/644 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7107 - loss: 0.5804 - val_accuracy: 0.7285 - val_loss: 0.5543
Epoch 2/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7205 - loss: 0.5618 - val_accuracy: 0.7353 - val_loss: 0.5479
Epoch 3/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7203 - loss: 0.5579 - val_accuracy: 0.7310 - val_loss: 0.5499
Epoch 4/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7311 - loss: 0.5471 - val_accuracy: 0.7349 - val_loss: 0.5504
Epoch 5/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7327 - loss: 0.5487 - val_accuracy: 0.7339 - val_loss: 0.5447
Epoch 6/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7323 - loss: 0.5483 - val_accuracy: 0.7331 - val_loss: 0.5467
Epoch 7/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7351 - loss: 0.5478 - val_accuracy: 0.7316 - val_loss: 0.5456
Epoch 8/200
644/644 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7280 - loss: 0.5493 - val_accuracy: 0.7394

In [18]:
#Evaluate Model
loss, accuracy = nn_optimized.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Loss: {loss}, Accuracy: {accuracy}")


Loss: 0.577116072177887, Accuracy: 0.7304956316947937
